# TP 2 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [ ]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

### Récupération des données

In [ ]:
function readKnaptxtInstance(filename)
    price=Int64[]
    weight=Int64[]
    KnapCap=Int64[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [ ]:
function testSondability_LP(capacity, node_profit, borne, BestProfit)
    TA, TO, TR = false, false, false
    
    if capacity < 0 #Test d'admissibilité
        TA = true
        println("TA")
    elseif borne + node_profit <= BestProfit #Test d'optimalite
        TO = true
        println("TO")
    elseif capacity == 0 #Test de resolution
        TR = true
        println("TR")
    else
        println("non sondable")
    end
    
    TA, TO, TR
end

## Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [ ]:
function separateNodeThenchooseNext_lexicographic_depthfirst!(listobjs, listvals, n)
    # this node is non-sondable. Apply the branching criterion to separate it into two subnodes
    # and choose the child-node at the left

    # lexicographic branching criterion: branch on the 1st object not yet fixed
    i, obj = 1, 0
    while((i <= n) && (obj==0))
        if(!(i in listobjs))
            obj=i
        end
        i+=1
    end

    println("\nbranch on object ", obj, "\n")

    # depthfirst exploration strategy: the node selected will be the most left of the child-nodes just created
    push!(listobjs,obj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
end


function exploreNextNode_depthfirst!(listobjs, listvals, listnodes)
    #this node is sondable, go back to parent node then right child if possible

    stop=false
    #check if we are not at the root node
    if (length(listobjs)>= 1)
        #go back to parent node
        obj=pop!(listobjs)
        theval=pop!(listvals)
        tmp=pop!(listnodes)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
        end
        if theval==1.0
            push!(listobjs,obj)
            push!(listvals,0.0)
        else
            println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        println("\nFINISHED")
        stop=true
    end
    return stop
end

###  Création de la relaxation linéaire (= modèle associé au noeud 0): <span style="color:red"> SECTION A SUPPRIMER !!!! </span>

<span style="color:red"> Cette section est à commenter/supprimer et remplacer par vos propres calculs de bornes supérieures et autres, par exemple basées sur les bornes 1 et 2 vues en cours, ou d'autres calculs de bornes de votre choix/conception validés au préalable par votre encadrant/e de TP </span>

In [ ]:
function borne1(price, weight, capacity)
    maxRatio = 0
    for i in 1:length(weight)
        ratio = price[i] / weight[i]
        if ratio > maxRatio
            maxRatio = ratio
        end
    end
    return capacity * maxRatio
end

function borne2(price, weight, capacity)
    res = 0

    if length(price) > 0

        # Liste de paires Ratio, Poids
        liste  = []
        for i in 1:length(price)
            ratio = price[i] / weight[i]
            liste = [liste; (ratio, weight[i])]
        end

        # Tri en fonction du ratio (x[1] ici)
        sort!(liste, by=x->x[1], rev=true)

        # Remplissage du sac
        for i in 1:length(liste)
            # Ajoute des objets avec les meilleurs ratios
            # Et Ajout d'une fraction du dernier object
            while ((capacity - liste[i][2]) > 0)
                res += liste[i][1] * liste[i][2]
                capacity -= liste[i][2]
            end
        end
    end

    return res
end

function borne(price, weight, capacity, borne_n)
    if borne_n == 1
        return borne1(price, weight, capacity)
    elseif borne_n == 2
        return borne2(price, weight, capacity)
    end
end

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [ ]:
function solveKnapInstance(filename, borne_n)

    compteur_node = 0

    price, weight, capacity = readKnaptxtInstance(filename)

    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree

    #intermediate structure to navigate in the search tree
    listobjs=Int64[]
    listvals=Float64[]
    listnodes=Int64[]

    BestProfit::Float64=-1.0
    Bestsol=Float64[]

    current_node_number::Int64=0
    stop = false

    while(!stop)

        compteur_node += 1

        println("\nNode number ", current_node_number, ": \n---------------\n")

        #Update the graphical tree
        push!(trNamenodes,current_node_number+1) 
        if(length(trNamenodes)>=2)
            push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
            push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
        end
        push!(listnodes, current_node_number)

        ## Mise à jour des prix, des poids, de la capacité, du profit
        node_price = deepcopy(price)
        node_weight = deepcopy(weight)
        node_capacity = capacity
        node_profit = 0
        for i in 1:length(listobjs)
            if listvals[i] == 1.0 # si on met cet objet dans le sac ...
                node_capacity -= weight[listobjs[i]] # retire le poids de l'objet à la capacité
                node_profit += price[listobjs[i]] # ajoute le prix de l'objet au profit
            end
        end
        # on retire les objets déjà pris
        deleteat!(node_price, listobjs)
        deleteat!(node_weight, listobjs)

        ## Mise à jour éventuelle de la meilleure solution
        if node_capacity >= 0 # on doit ne pas vérifier le TA
            if node_profit > BestProfit
                if node_profit >= BestProfit
                    println("\nPrevious Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
                    BestProfit = node_profit
                    Bestsol = zeros(length(price))
                    for i in 1:length(listobjs)
                        Bestsol[listobjs[i]] = listvals[i]
                    end
                    println("\nNew Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
                end
            end
        end

        ## Calcul de la borne
        node_borne = borne(node_price, node_weight, node_capacity, borne_n)

        ## Tests de sondabilité
        TA, TO, TR = testSondability_LP(node_capacity, node_profit, node_borne, BestProfit)
        is_node_sondable = TA || TO || TR
        if(!is_node_sondable)
            separateNodeThenchooseNext_lexicographic_depthfirst!(listobjs, listvals, length(price))
        else
            stop = exploreNextNode_depthfirst!(listobjs, listvals, listnodes)
        end

        ## Progression dans l'arbre
        current_node_number = current_node_number + 1
    end

    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)
    println("Nombre de noeuds explorés : ", compteur_node)


    return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes

end


### Affichage du résultat final

In [ ]:
function solveNdisplayKnap(filename, borne_n, show_graph)

    println("\n Branch-and-Bound for solving a knapsack problem. \n\n Solving instance '" * filename * "'\n")

    BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = solveKnapInstance(filename, borne_n)

    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    if show_graph
        println("\n Branch-and-bound tree visualization : start display ...")
        display(graphplot(trParentnodes, trChildnodes, names=trNamenodes, method=:tree))
        println("... end display. \n\n")
    end

end

In [ ]:
INSTANCES = ["InstancesKnapSack/test.opb.txt", "InstancesKnapSack/similar_weights/knapPI_9_50_1000_2_-997.opb.txt", "InstancesKnapSack/profit_ceiling/knapPI_15_20_1000_1_-999.opb.txt", "InstancesKnapSack/weakly_correlated_span/knapPI_12_20_1000_1_-970.opb.txt"]
INSTANCE_N = 4         # entre 1 et 4, pour lancer différents tests
BORNE_N = 2             # borne 1 ou borne 2
SHOW_GRAPH = false      # afficher le graph ?

solveNdisplayKnap(INSTANCES[INSTANCE_N], BORNE_N, SHOW_GRAPH)

# Questions Préliminaires:

### 1. Règle de Séparation :
   La séparation suit la règle d'ordre lexicographique et on sélectionne le premier objet disponible dans la liste (c'est à dire le premier objet dans la liste qui n'a pas été encore pris).

### 2. Calcul de la Borne Supérieure :
   La borne supérieure est déterminée en résolvant la relaxation linéaire : on attribue la valeur 1 aux objets pris et 0 à ceux non pris.

### 3. Tests de Sondabilité TA, TO, TR :
   - TA : Le test d'admissibilité est validé lorsque la relaxation linéaire possède une solution.
   - TO : Le test d'optimalité est validé lorsque qu'il existe une solution inférieure au profit maximal (la borne supérieure).
   - TR : Le test de résolution est validé lorsque qu'une de ces solution est atteinte pour des valeurs entières.

### 4. Stratégie d’Exploration :
   La stratégie d'exploration est la stratégie depthfirst : c'est une exploration en profondeur. On va à gauche jusqu'à l'arrêt puis on remonte.

# Code et analyse :

### 1. Borne 1 :
   Voir le code commenté de la fonction _borne1_, partie "Création de la relaxation linéaire", 

### 2. Adaptation Branch & Bound :
   Voir le code commenté des parties "Tests de sondabilités" et "Boucle principale"

### 3. Borne 2 :
   Voir le code commenté de la fonction _borne2_, partie "Création de la relaxation linéaire", 

### 4. Points clés de notre implémentation :

  - Règle de séparation :
    nous conservons la règle de séparation par ordre lexicographique

  - Tests de sondabilité :
      - TA : le test d'admissibilité réussit si la capacité du sac à dos est négative, la relaxation linéaire est impossible car les contraintes ne peuvent pas être satisfaites.
      - TO : le test d'optimalité réussit si la valeur de la borne est pire que la meilleure solution connue.
      - TR : le test de relaxation réussit si la capacité du sac à dos est nulle, c'est à dire que la solution est atteinte pour des valeurs entières car on ne peux pas ajouter de fraction de l'objet suivant.

  - Stratégie d'exploration :
    nous conservons la stratégie d'exploration en profondeur

  - Choix de la structure de données :
  Nous avons conservé *BestProfit* et *Bestsol* et ces valeurs sont mises à jour lorsque la meilleure solution change.
  Nous avons crée les variables *node_price*, *node_weight* et *node_capacity* pour qu'à chaque itération on puisse calculer les bornes.

### 5. Comparaison des performances :

Pour ces tests, nous avons choisis quatre jeux de données de natures différentes.

**Borne 1**

| Instance         | 1 | 2 | 3 | 4 | 
|-------------------|---|---|---|---|
| Nombre de noeuds explorés | 17 | 1255 | 1819 | 117 | 
| Temps d'exécution (ms) | 21 | 714 | 748 | 82 | 
| Solution optimale | 65 | 997 | 999 | 970 | 
| Erreur | - | - | - | - |

**Borne 2**

| Instance         | 1 | 2 | 3 | 4 | 
|-------------------|---|---|---|---|
| Nombre de noeuds | 9 | 139 | 839 | 29 | 
| Temps d'exécution (ms) | 19 | 257 | 444 | 42 | 
| Solution optimale | 65 | 997 | 996 | 880 | 
| Erreur | - | - | 0.3% | 9.3% |


Tout d'abord on remarque que la borne 2 permet une résolution plus efficace et plus rapide du problème, en explorant moins de noeuds que la borne 1.

La borne 2 est donc plus performante que ce soit en temps d'exécution et en nombre de noeuds explorés (proportionnellement).

Cependant nous remarquons que la borne 2 peut mener à des résultats incorrects selon les données fournies. Cette erreur peut aller de l'imprécision (instance 3) à un écart beaucoup plus important (instance 4). Il faut donc être prudent dans son utilisation.

### 6. Améliorations possibles :
Pour améliorer cet algorithme de Branch & Bound nous envisagerions de changer la stratégie d'exploration ce qui pourrait améliorer les performances de la recherche de solution optimale dans certains cas. Nous pourrions par exemple implémenter un parcours en largeur à l'aide de file (cela pourrait permettre un exploration plus équilibrée de l'arbre).